In [37]:
from typing import List
from abc import abstractmethod, ABC

class Action:
    pass

class InitAction(Action):
    def __init__(self, arr: List[int]):
        self.arr = [x for x in arr]

    def __repr__(self):
        return f'Init({self.arr})'

class CompareAction(Action):
    def __init__(self, i: int , j: int):
        self.i = i
        self.j = j

    def __repr__(self):
        return f'Compare({self.i}, {self.j})'

class SwapAction(Action):
    def __init__(self, i: int , j: int):
        self.i = i
        self.j = j

    def __repr__(self):
        return f'Swap({self.i}, {self.j})'


In [38]:
class SortingAlgorithm(ABC):
    def __init__(self, arr: List[int]):
        self.arr = arr
        self.actions: List[Action] = [InitAction(arr)]

    def _cmp(self, i: int, j: int) -> int:
        self.actions.append(CompareAction(i, j))
        return self.arr[i] - self.arr[j]

    def _swap(self, i: int, j: int) -> None:
        self.arr[i], self.arr[j] = self.arr[j], self.arr[i]
        self.actions.append(SwapAction(i, j))

    @abstractmethod
    def sort(self) -> None:
        raise NotImplemented()

    def getActions(self) -> List[Action]:
        return self.actions

    def getArrayStates(self) -> List[List[int]]:
        states = []
        for action in self.actions:
            if isinstance(action, InitAction):
                states.append([x for x in action.arr])
            elif isinstance(action, SwapAction):
                prevState = states[-1]
                states.append([x for x in prevState])
                # Perform the swap on the last state
                states[-1][action.i], states[-1][action.j] = states[-1][action.j], states[-1][action.i]
            elif isinstance(action, CompareAction):
                pass
        return states


In [39]:
class BubbleSort(SortingAlgorithm):
    def sort(self):
        for i in range(len(self.arr)):
            for j in range(i + 1, len(self.arr)):
                if self._cmp(i, j) > 0:
                    self._swap(i, j)


In [40]:
class MergeSort(SortingAlgorithm):
    def sort(self, l, r):
        if l < r:
            m = l + (r - l) // 2
            self.sort(l, m)
            self.sort(m + 1, r)
            self.merge(l, m, r)

    def merge(self, l, m, r):
        n1 = m - l + 1
        n2 = r - m

        # create temp arrays
        L = []
        R = []
        # Copy data to temp arrays L[] and R[]
        for i in range(0, n1):
            L.append({
                "value": self.arr[l + i],
                "index": l + i
            })

        for j in range(0, n2):
            R.append({
                "value": self.arr[m + 1 + j],
                "index": m + 1 + j
            })

        # Merge the temp arrays back into arr[l..r]
        i = 0     # Initial index of first subarray
        j = 0     # Initial index of second subarray
        k = l     # Initial index of merged subarray

        while i < n1 and j < n2:
            if self._cmp(L[i]["index"], R[j]["index"]) <= 0:
                if k != L[i]["index"]:
                    for item in L + R:
                        if item["value"] == self.arr[k]:
                            item["index"] = L[i]["index"]
                    self._swap(k, L[i]["index"])
                    L[i]["index"] = k
                i += 1
            else:
                if k != R[j]["index"]:
                    for item in L + R:
                        if item["value"] == self.arr[k]:
                            item["index"] = R[j]["index"]
                    self._swap(k, R[j]["index"])
                    R[j]["index"] = k
                j += 1
            k += 1

        # Copy the remaining elements of L[], if there
        # are any
        while i < n1:
            if k != L[i]["index"]:
                self._swap(k, L[i]["index"])
            i += 1
            k += 1

        # Copy the remaining elements of R[], if there
        # are any
        while j < n2:
            if k != R[j]["index"]:
                self._swap(k, R[j]["index"])
            j += 1
            k += 1


In [41]:
import matplotlib.pyplot as plt
from pathlib import Path

def createFrames(location: Path, sortAlg: SortingAlgorithm) -> None:
    if not location.exists():
        location.mkdir(parents=True, exist_ok=True)

    states = sortAlg.getArrayStates()

    for index, state in enumerate(states):
        plt.bar(range(len(state)), state)
        plt.axis('off')
        plt.savefig(f'{location}/frame_{index}.png')
        plt.clf()


In [42]:
if __name__ == '__main__':
    arr = [2, 3, 5, 4, 1]
    bubble = BubbleSort([x for x in arr])
    bubble.sort()
    print(f"Bubble sort:\n{bubble.getActions()}")
    for state in bubble.getArrayStates():
        print(state)
    createFrames(Path('output/bubble'), bubble)
    print("=====")

    merge = MergeSort([x for x in arr])
    merge.sort(0, len(arr) - 1)
    print(f"Merge sort:\n{merge.getActions()}")
    for state in merge.getArrayStates():
        print(state)
    createFrames(Path('output/merge'), bubble)
    print("=====")


Bubble sort:
[Init([2, 3, 5, 4, 1]), Compare(0, 1), Compare(0, 2), Compare(0, 3), Compare(0, 4), Swap(0, 4), Compare(1, 2), Compare(1, 3), Compare(1, 4), Swap(1, 4), Compare(2, 3), Swap(2, 3), Compare(2, 4), Swap(2, 4), Compare(3, 4), Swap(3, 4)]
[2, 3, 5, 4, 1]
[1, 3, 5, 4, 2]
[1, 2, 5, 4, 3]
[1, 2, 4, 5, 3]
[1, 2, 3, 5, 4]
[1, 2, 3, 4, 5]
=====
Merge sort:
[Init([2, 3, 5, 4, 1]), Compare(0, 1), Compare(0, 2), Compare(1, 2), Compare(3, 4), Swap(3, 4), Compare(0, 3), Swap(0, 3), Compare(3, 4), Swap(1, 3), Compare(3, 4), Swap(2, 3), Compare(3, 4), Swap(3, 4)]
[2, 3, 5, 4, 1]
[2, 3, 5, 1, 4]
[1, 3, 5, 2, 4]
[1, 2, 5, 3, 4]
[1, 2, 3, 5, 4]
[1, 2, 3, 4, 5]
=====


<Figure size 640x480 with 0 Axes>